In [ ]:
!pip install pyzill

In [2]:
import pyzill
import json
import geopandas as gpd
# Load the GeoJSON file
gdf = gpd.read_file('/Users/maples/GitHub/Zillow-Scrape/grt_buffer_bbox_wgs84.geojson')

# Calculate the bounding box
bounds = gdf.total_bounds
sw_long, sw_lat, ne_long, ne_lat = bounds
zoom_value = 2
#pagination is for the list that you see at the right when searching
#you don't need to iterate over all the pages because zillow sends the whole data on mapresults at once on the first page
#however the maximum result zillow returns is 500, so if mapResults is 500
#try playing with the zoom or moving the coordinates, pagination won't help because you will always get at maximum 500 results
pagination = 1 
results_sold = pyzill.sold(pagination, ne_lat,ne_long,sw_lat,sw_long,zoom_value, "")
results_sale = pyzill.for_sale(pagination, ne_lat,ne_long,sw_lat,sw_long,zoom_value, "")
results_rent = pyzill.for_rent(pagination, ne_lat,ne_long,sw_lat,sw_long,zoom_value, "")
jsondata_sold = json.dumps(results_sold)
jsondata_sale = json.dumps(results_sale)
jsondata_rent = json.dumps(results_rent)
f = open("./jsondata_sold.json", "w")
f.write(jsondata_sold)
f.close()
f = open("./jsondata_sale.json", "w")
f.write(jsondata_sale)
f.close()
f = open("./jsondata_rent.json", "w")
f.write(jsondata_rent)
f.close()

In [ ]:
import json
import os

def convert_to_geojson(input_file, output_file):
    # Read the JSON data from the input file
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    # Initialize the GeoJSON structure
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }
    
    # Loop through the list of properties and extract relevant data
    for property in data.get("listResults", []):
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    property["latLong"]["longitude"],
                    property["latLong"]["latitude"]
                ]
            },
            "properties": {
                "zpid": property["zpid"],
                "price": property["price"],
                "address": property["address"],
                "beds": property.get("beds", None),
                "baths": property.get("baths", None),
                "area": property.get("area", None),
                "statusText": property.get("statusText", None),
                "detailUrl": property.get("detailUrl", None)
            }
        }
        geojson["features"].append(feature)
    
    # Write the GeoJSON data to the output file
    with open(output_file, 'w') as f:
        json.dump(geojson, f, indent=2)

# Define the input and output file paths
input_file = "/Users/maples/GitHub/Zillow-Scrape/jsondata_sale.json"
output_file = "/Users/maples/GitHub/Zillow-Scrape/output_properties.geojson"

# Convert the JSON data to GeoJSON format
convert_to_geojson(input_file, output_file)

print(f"GeoJSON file '{output_file}' created successfully.")